In [1]:

import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense,Embedding, LSTM, Bidirectional,GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
import nltk
import re
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('fake-news.csv')

In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data.shape

(20800, 5)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,20800.0,10399.500000,6004.587135,0.0,5199.75,10399.5,15599.25,20799.0
label,20800.0,0.500625,0.500012,0.0,0.00,1.0,1.00,1.0


In [7]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df=data.dropna()

In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
df.duplicated().sum()


0

In [11]:
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [12]:
X=df.drop('label',axis=1)
y=df['label']

In [13]:

sentences=X.copy()
sentences.reset_index(inplace=True)
ps = PorterStemmer()
corpus = []
for i in range(0,len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

Now we will one hot encode the data as we have word list and we will get the index w.r.t vocab_size.One-hot encodes a text into a list of word indexes of size n.<br>
Refer:https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/one_hot

In [14]:
vocab_size = 15000
onehot = [one_hot(words, vocab_size)for words in corpus]
onehot[:10]

[[4812, 5283, 6358, 14230, 4165, 3705, 1453, 9691, 9710, 1436],
 [10333, 11437, 12958, 8815, 12241, 3557, 5083],
 [13615, 13778, 8500, 10116],
 [5286, 13710, 3037, 11450, 714, 4388],
 [2643, 12241, 3838, 7530, 5927, 13576, 12241, 8489, 1503, 4735],
 [8495,
  5438,
  5690,
  1745,
  361,
  9158,
  4707,
  4908,
  13691,
  6710,
  5560,
  12665,
  13331,
  5892,
  5083],
 [4680, 5636, 12501, 3995, 1873, 14209, 3002, 6598, 9239, 12421, 222],
 [9760, 11436, 626, 12278, 13228, 488, 9158, 3960, 9239, 12421, 222],
 [2297, 13844, 3065, 6282, 9157, 10073, 7940, 286, 9158, 282],
 [7682, 1626, 3115, 5102, 5478, 1041, 9234, 12369]]

In [15]:
length = 75
embedding=pad_sequences(onehot, maxlen=length, padding='pre')
embedding[:10]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,  4812,  5283,  6358, 14230,  4165,  3705,  1453,
         9691,  9710,  1436],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     

In [16]:
embedding_vector_features=40 
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features, input_length=length))
model.add(Bidirectional(LSTM(200)))
#model.add(GRU(200))
#model.add(LSTM(200))
model.add(Dense(1,activation='sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [18]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 75, 40)            600000    
                                                                 
 gru (GRU)                   (None, 200)               145200    
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 745,401
Trainable params: 745,401
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
X=np.array(embedding)
Y=np.array(y)

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=1)
model.fit(X_train,y_train,epochs=10,batch_size=64,validation_data=(X_test,y_test) )

Epoch 1/10
200/200 [==============================] - 5s 13ms/step - loss: 0.2815 - accuracy: 0.8619 - val_loss: 0.1776 - val_accuracy: 0.9264
Epoch 2/10
200/200 [==============================] - 2s 10ms/step - loss: 0.1105 - accuracy: 0.9585 - val_loss: 0.1806 - val_accuracy: 0.9269
Epoch 3/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0572 - accuracy: 0.9800 - val_loss: 0.2289 - val_accuracy: 0.9285
Epoch 4/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0252 - accuracy: 0.9927 - val_loss: 0.2908 - val_accuracy: 0.9302
Epoch 5/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0117 - accuracy: 0.9970 - val_loss: 0.3569 - val_accuracy: 0.9227
Epoch 6/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.3882 - val_accuracy: 0.9202
Epoch 7/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0041 - accuracy: 0.9988 - val_loss: 0.4280 - val_accuracy: 0.9214

In [21]:
y_pred=model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)

y_pred

172/172 [==============================] - 1s 3ms/step


array([[1],
       [1],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [22]:
cm=confusion_matrix(y_test, y_pred) 
print(cm)


[[2903  223]
 [ 213 2147]]


In [23]:
acc=print(accuracy_score(y_test, y_pred))
acc

0.9205249726576741
